Direct link from kaggle
https://www.kaggle.com/datasets/heesoo37/120-years-of-olympic-history-athletes-and-results?resource=download

In [2]:
import pandas as pd

In [5]:
athletes = pd.read_csv('Data/athlete_events.csv')

In [7]:
athletes.head(3)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN


In [8]:
athletes = athletes[athletes['Season'] == 'Summer']

In [10]:
def team_summary(data):
    return pd.Series({
        'team': data.iloc[0,:]["NOC"],
        'country': data.iloc[-1,:]["Team"],
        'year': data.iloc[0,:]["Year"],
        'events': len(data['Event'].unique()),
        'athletes': data.shape[0],
        'age': data["Age"].mean(),
        'height': data['Height'].mean(),
        'weight': data['Weight'].mean(),
        'medals': sum(~pd.isnull(data["Medal"]))
    })

team = athletes.groupby(["NOC", "Year"]).apply(team_summary)

In [11]:
team = team.reset_index(drop=True)
team = team.dropna()

In [12]:
team.head(3)

,team,country,year,events,athletes,age,height,weight,medals
3,AFG,Afghanistan,1960,13,16,23.3125,170.6875,69.4375,0
4,AFG,Afghanistan,1964,8,8,22.0000,161.0000,64.2500,0
5,AFG,Afghanistan,1968,5,5,23.2000,170.2000,70.0000,0


In [13]:
def prev_medals(data):
    data = data.sort_values("year", ascending=True)
    data["prev_medals"] = data["medals"].shift(1)
    data["prev_3_medals"] = data.rolling(3, closed="left", min_periods=1).mean()["medals"]
    return data

team = team.groupby(["team"]).apply(prev_medals)
team = team.reset_index(drop=True)
team = team[team["year"] > 1960]
team = team.round(1)

/var/folders/sz/fwzwqnk95yd56ss21x7qvsd00000gn/T/ipykernel_12595/42556474.py:4: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['country', 'team'], dtype='object')
  data["prev_3_medals"] = data.rolling(3, closed="left", min_periods=1).mean()["medals"]
/var/folders/sz/fwzwqnk95yd56ss21x7qvsd00000gn/T/ipykernel_12595/42556474.py:4: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['country', 'team'], dtype='object')
  data["prev_3_medals"] = data.rolling(3, closed="left", min_periods=1).mean()["medals"]
/var/folders/sz/fwzwqnk95yd56ss21x7qvsd00000gn/T/ipykernel_12595/42556474.py:4: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a 

In [14]:
team.to_csv("/Data/teams.csv", index=False)